# Workshop Developing AI Agents with AMD GPUs: Build Your OpenManus Agent with MCPs using vLLM, and AMD MI300X GPU

Welcome to this hands-on workshop! Throughout this tutorial, we'll leverage AMD GPUs and **Model Context Protocol (MCP)** ,an open standard for exposing LLM tools via API, to deploy powerful language models like Qwen3. Key components:
- 🖥️ **vLLM** for GPU-optimized inference
- 🛠️ **OpenManus** for agent/tool management
- 🔌 **MCP Servers** for pre-built tool integration

You'll learn how to set up your environment, deploy large language models like Qwen3, connect them to real-world tools using MCP, and build a conversational agent capable of reasoning and taking actions.

By the end of this workshop, you’ll have built an AI-powered Airbnb assistant agent—one that can find a place to stay based on your preferences like location, budget, and travel dates.

Let’s dive in!

## Table of Contents

- [Step 1: Launching vLLM Server on AMD GPUs](#step1)
- [Step 2: Installing Dependencies](#step2)
- [Step 3: Create a simple instance of OpenManus](#step3)
- [Step 4: Tsing MCP server for OpenManus Agent](#step4)
- [Step 5: Challenge with Prize](#step5)

<a id="step1"></a>

## Step 1: Launch a vLLM Server

In this workshop we are going to use [vLLM](https://github.com/vllm-project/vllm) as our inference serving engine. vLLM provides many benefits such as fast model execution, extensive list of supported models, easy to use, and best of all it's open-source. 

### Deploy Qwen3-30B-A3B Model with vLLM

In this workshop we have already configured `VLLM_PORT` for you. Let's verify the port number.


In [ ]:
!echo $VLLM_PORT


Time to start your vLLM server and creating an end-point for your LLM. Let's open a terminal using your Jupyter server. Then run the following command in this terminal to start the vLLM server:

```bash
VLLM_USE_TRITON_FLASH_ATTN=0 \
vllm serve Qwen/Qwen3-30B-A3B \
    --served-model-name Qwen3-30B-A3B \
    --api-key abc-123 \
    --port $VLLM_PORT \
    --enable-auto-tool-choice \
    --tool-call-parser hermes \
    --trust-remote-code
```

Open another terminal and monitor the GPU utilization by running this command:

```bash
watch rocm-smi
```

Upon successful launch, your server should be accepting incoming traffic through an OpenAI-compatible API. Let's set some environment variables for our server so we can use throughout this tutorial:

In [ ]:
import os

BASE_URL = f"http://localhost:{os.environ['VLLM_PORT']}/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

We can verify your model is available at the `BASE_URL` we just set by running the following command.

In [ ]:
!curl http://localhost:$VLLM_PORT/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

Congratulations, you now just launched a powerful server that can serve any incoming request and allowing you to build amazing applications. Wasn't that easy?🎉 

<a id="step2"></a>

## Step 2: Installing Dependencies

We are going to use `OpenManus`. Let's install the dependencies:

In [1]:
!git clone https://github.com/FoundationAgents/OpenManus.git

Cloning into 'OpenManus'...
remote: Enumerating objects: 3330, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 3330 (delta 38), reused 13 (delta 13), pack-reused 3268 (from 3)
Receiving objects: 100% (3330/3330), 3.78 MiB | 574.00 KiB/s, done.
Resolving deltas: 100% (2006/2006), done.


In [ ]:
!conda create -n open_manus-2 python=3.12 -y

/opt/anaconda3/lib/python3.12/site-packages/conda/base/context.py:198: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.html

  deprecated.topic(
/opt/anaconda3/lib/python3.12/site-packages/conda/base/context.py:198: FutureWarning: Adding 'defaults' to channel list implicitly is deprecated and will be removed in 25.3. 

To remove this warning, please choose a default channel explicitly with conda's regular configuration system, e.g. by adding 'defaults' to the list of channels:

  conda config --add channels defaults

For more information see https://docs.conda.io/projects/conda/en/stable/user-guide/configuration/use-condarc.

In [7]:
!pip install -r OpenManus/requirements.txt
!playwright install


<a id="step3"></a>

## Step 3: Create a simple instance of OpenManus Agent

Let's start by creating a custom OpenAI Compatible endpoint for our agent and adding more configuration. 


In [65]:
%env BASE_URL=http://64.139.222.210:8000/v1 # This is the vLLM server we started earlier
%env API_KEY=abc-123  # This is the API key we set earlier
%env MODEL=Qwen3-30B-A3B # This is the model we are using
%env HEADLESS=true # This is the headless mode we are using

env: BASE_URL=http://64.139.222.210:8000/v1
env: API_KEY=abc-123
env: MODEL=Qwen3-30B-A3B
env: HEADLESS=true


Now Define the task run by the OpenManus

In [37]:
task_prompt = "I am at San Jose, I want to make a mapo tofu, plese instuct me what should I buy all the ingredientsat what stores and how to do it"

Great! now that we have the basics of creating an agent instance, and connecting it to the model we started serving with vLLM earlier.

Let's test the agent.

In [67]:
!python OpenManus/main.py --prompt "{task_prompt}"

env_base_url: http://64.139.222.210:8000/v1
env_api_key: abc-123
MODEL: Qwen3-30B-A3B
default_settings: {'model': 'Qwen3-30B-A3B', 'base_url': 'http://64.139.222.210:8000/v1', 'api_key': 'abc-123', 'max_tokens': 4096, 'max_input_tokens': None, 'temperature': 0.0, 'api_type': '', 'api_version': ''}
INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
2025-06-03 07:12:53.123 | WARNING  | __main__:main:23 - Processing your request...
2025-06-03 07:12:53.123 | INFO     | app.agent.base:run:140 - Executing step 1/20
2025-06-03 07:13:00.857 | INFO     | app.llm:update_token_count:243 - Token usage: Input=1904, Completion=711, Cumulative Input=1904, Cumulative Completion=711, Total=2615, Cumulative Total=2615
2025-06-03 07:13:00.858 | INFO     | app.agent.toolcall:think:81 - ✨ Manus's thoughts: <think>
Okay, the user is in San Jose and wants to make ma

An example of failure case using OpenManus

In [68]:
task_prompt2 = "I am at San Jose, I want to book a flight to NewYork today, please help me to find the best flight and the best price"


In [73]:
!python OpenManus/main.py --prompt "{task_prompt2}"

env_base_url: http://64.139.222.210:8000/v1
env_api_key: abc-123
MODEL: Qwen3-30B-A3B
default_settings: {'model': 'Qwen3-30B-A3B', 'base_url': 'http://64.139.222.210:8000/v1', 'api_key': 'abc-123', 'max_tokens': 4096, 'max_input_tokens': None, 'temperature': 0.0, 'api_type': '', 'api_version': ''}
INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
2025-06-03 07:30:37.043 | WARNING  | __main__:main:23 - Processing your request...
2025-06-03 07:30:37.044 | INFO     | app.agent.base:run:140 - Executing step 1/20
2025-06-03 07:30:41.153 | INFO     | app.llm:update_token_count:243 - Token usage: Input=1825, Completion=367, Cumulative Input=1825, Cumulative Completion=367, Total=2192, Cumulative Total=2192
2025-06-03 07:30:41.153 | INFO     | app.agent.toolcall:think:81 - ✨ Manus's thoughts: <think>
Okay, the user is in San Jose and wants to book a 



<a id="step4"></a>

## Step 4: Adding a MCP server

Now that we learned how to create a custom tool and provide the agent access to this tool. Let's now explore a trendy topic of [Model Context Protocol](https://modelcontextprotocol.io/introduction). We are going to explore how we can replace our custom tool with a simple MCP server that can serve our agent and provide similar information.

**Why MCP?** MCP servers provide:
- ✅ Standardized API interfaces
- 🔄 Reusable across projects
- 📦 Pre-built functionality

Let's replace our custom time tool with an official MCP time server:

### Installing Time MCP Server

We are going to start by installing this MCP server:


In [77]:
!pip install -q mcp-server-time

Creating the MCP information file

In [80]:
mcp_info = {
  "mcpServers": {
    "time": {
      "type": "stdio",
      "command": "python",
      "args": ["-m", "mcp_server_time", "--local-timezone=America/New_York"]
    }
  }
}

In [81]:
## create the mcp.json under OpenManus/config/mcp.json
import json
import os
# Create config directory if it doesn't exist
os.makedirs("OpenManus/config", exist_ok=True)
# Write mcp.json file
with open("OpenManus/config/mcp.json", "w") as f:
    json.dump(mcp_info, f, indent=2)

Great, let's see if the agent can use the MCP to give us the correct time now.

In [79]:
!python OpenManus/main.py --prompt "Tell me the time in San Francisco"

env_base_url: http://64.139.222.210:8000/v1
env_api_key: abc-123
MODEL: Qwen3-30B-A3B
default_settings: {'model': 'Qwen3-30B-A3B', 'base_url': 'http://64.139.222.210:8000/v1', 'api_key': 'abc-123', 'max_tokens': 4096, 'max_input_tokens': None, 'temperature': 0.0, 'api_type': '', 'api_version': ''}
INFO     [browser_use] BrowserUse logging setup complete with level info
INFO     [root] Anonymized telemetry enabled. See https://docs.browser-use.com/development/telemetry for more information.
2025-06-03 07:54:19.463 | INFO     | app.tool.mcp:_initialize_and_list_tools:124 - Connected to server time with tools: ['get_current_time', 'convert_time']
2025-06-03 07:54:19.464 | INFO     | app.agent.manus:initialize_mcp_servers:85 - Connected to MCP server time using command python
2025-06-03 07:54:19.464 | WARNING  | __main__:main:23 - Processing your request...
2025-06-03 07:54:19.464 | INFO     | app.agent.base:run:140 - Executing step 1/20
2025-06-03 07:54:22.108 | INFO     | app.llm:update_


Tadaa! Now you have officially used an MCP server to power-up your agent. In the next section we show how you can your turn many ideas into real working projects by using 100s of free or paid MCP servers available today.



<a id="step6"></a>

## Step 5: Turn your agent to Multi-MCP user

As we experience in the last section, MCP servers are really easy to use and they provide a standard way of providing LLMs the tools we need. There are already thousands of MCP servers available for us to use. There are some MCP trackers that you can always use to find out about available servers. Here are some for your reference:
- https://github.com/modelcontextprotocol/servers
- https://mcp.so/




In this part of the workshop we are going to build an agent that can help you browse available Airbnbs to book. We can now build on top of what we have so far and add an open-source Airbnb MCP server to our agent. To do so, let's start by defining our Airbnb server.

In [109]:
!pip install mcp-weather

ERROR: Could not find a version that satisfies the requirement mcp-weather (from versions: none)
ERROR: No matching distribution found for mcp-weather


In [114]:
mcp_info = {
  "mcpServers": {
    "time": {
      "type": "stdio",
      "command": "python",
      "args": [
        "-m",
        "mcp_server_time",
        "--local-timezone=America/New_York"
      ]
    },
    "weather": {
        "type": "stdio",
        "command": "uv",
        "args": [
            "--directory",
            "/Users/zhukunlun/Documents/GitHub/amd-gpu-workshops/notebooks/OpenManus/weather",
            "run",
            "weather.py"
        ]
    }
  }
}

In [115]:
## create the mcp.json under OpenManus/config/mcp.json
import json
import os
# Write mcp.json file
with open("OpenManus/config/mcp.json", "w") as f:
    json.dump(mcp_info, f, indent=2)

Let's update our agent.

In [116]:
task_prompt = "Tell me the weather in San Francisco now"
!python OpenManus/main.py --prompt "{task_prompt}"


env_base_url: http://64.139.222.210:8000/v1
env_api_key: abc-123
MODEL: Qwen3-30B-A3B
default_settings: {'model': 'Qwen3-30B-A3B', 'base_url': 'http://64.139.222.210:8000/v1', 'api_key': 'abc-123', 'max_tokens': 4096, 'max_input_tokens': None, 'temperature': 0.0, 'api_type': '', 'api_version': ''}
Traceback (most recent call last):
  File "/Users/zhukunlun/Documents/GitHub/amd-gpu-workshops/notebooks/OpenManus/app/config.py", line 143, in load_server_config
    type=server_config["type"],
         ~~~~~~~~~~~~~^^^^^^^^
KeyError: 'type'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/zhukunlun/Documents/GitHub/amd-gpu-workshops/notebooks/OpenManus/main.py", line 4, in <module>
    from app.agent.manus import Manus
  File "/Users/zhukunlun/Documents/GitHub/amd-gpu-workshops/notebooks/OpenManus/app/agent/__init__.py", line 1, in <module>
    from app.agent.base import BaseAgent
  File "/Users/zhukunlun/Documents/GitHu

Finally, let's try our agent and see if it can browse through Airbnb listings.



<a id="step7"></a>

## Step 7: Challenge - Expand the Agent

**Task:** Add weather integration using an appropiate MCP server:
1. Launch weather MCP server
2. Add to agent's tools
3. Make agent suggest best travel dates based on weather

**Judging Criteria:**
✅ Functional weather integration
🎯 Logical tool selection
💡 Creative use of multiple tools
